In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('NorthBayVillage_ResiPipeline - Sheet1.csv')

In [4]:
df['geo_address'] = df['full_address'] + ' North Bay Village, FL'

In [5]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [6]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [7]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__5vd6p'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [8]:
df

,full_address,developers,land_purchase_price,units,description,story_link,geo_address,geocoded,lat,lon,image_links
0,1819-1855 79th Street in North Bay Village,Jesta Group,"$15,000,000",345,Jesta plans a 30-story multifamily and hotel t...,https://therealdeal.com/miami/2023/01/17/jesta...,1819-1855 79th Street in North Bay Village Nor...,"(25.8462074, -80.1539351)",25.846207,-80.153935,https://therealdeal.com/wp-content/uploads/202...
1,1555 North Bay Causeway,Sunbeam Properties,"$56,500,000","2,000",Sunbeam Properties plans a massive 7.3 million...,https://therealdeal.com/miami/2022/10/12/victo...,"1555 North Bay Causeway North Bay Village, FL","(25.8491, -80.15321399999999)",25.849100,-80.153214,https://therealdeal.com/wp-content/uploads/202...
2,1850 79th Street Causeway,Shoma Group,"$16,000,000",327,Shoma Group launched sales in September of Sho...,https://therealdeal.com/miami/2022/09/16/shoma...,"1850 79th Street Causeway North Bay Village, FL","(25.84776, -80.1473369)",25.847760,-80.147337,https://therealdeal.com/wp-content/uploads/202...
3,"8000, 8010 and 8020 East Drive",Macklowe Properties,"$47,700,000",NaN,An affiliate of Macklowe Properties paid $47.7...,https://therealdeal.com/miami/2023/05/19/mackl...,"8000, 8010 and 8020 East Drive North Bay Villa...","(25.8545286, -80.1575787)",25.854529,-80.157579,https://therealdeal.com/wp-content/uploads/202...
4,7918 West Drive,Pacific & Orient Properties,"$8,300,000",54,"Pacific & Orient Properties plans a 21-story, ...",https://therealdeal.com/tag/north-bay-village/,"7918 West Drive North Bay Village, FL","(25.850618, -80.160242)",25.850618,-80.160242,no_image


In [9]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[1]

'https://therealdeal.com/wp-content/uploads/2022/10/Main_north-bay-village-705x439.jpg'

In [10]:
df['image_links'].iloc[3]

'https://therealdeal.com/wp-content/uploads/2023/05/MIA-Macklowe-assembling-waterfront-land-in-North-Bay-Village-MAIN-r1-1024x640.jpg'

In [11]:
df['image_links'].iloc[4]

'no_image'

In [12]:
df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [13]:
df

,full_address,developers,land_purchase_price,units,description,story_link,geo_address,geocoded,lat,lon,image_links
0,1819-1855 79th Street in North Bay Village,Jesta Group,"$15,000,000",345,Jesta plans a 30-story multifamily and hotel t...,https://therealdeal.com/miami/2023/01/17/jesta...,1819-1855 79th Street in North Bay Village Nor...,"(25.8462074, -80.1539351)",25.846207,-80.153935,https://therealdeal.com/wp-content/uploads/202...
1,1555 North Bay Causeway,Sunbeam Properties,"$56,500,000","2,000",Sunbeam Properties plans a massive 7.3 million...,https://therealdeal.com/miami/2022/10/12/victo...,"1555 North Bay Causeway North Bay Village, FL","(25.8491, -80.15321399999999)",25.849100,-80.153214,https://therealdeal.com/wp-content/uploads/202...
2,1850 79th Street Causeway,Shoma Group,"$16,000,000",327,Shoma Group launched sales in September of Sho...,https://therealdeal.com/miami/2022/09/16/shoma...,"1850 79th Street Causeway North Bay Village, FL","(25.84776, -80.1473369)",25.847760,-80.147337,https://therealdeal.com/wp-content/uploads/202...
3,"8000, 8010 and 8020 East Drive",Macklowe Properties,"$47,700,000",NaN,An affiliate of Macklowe Properties paid $47.7...,https://therealdeal.com/miami/2023/05/19/mackl...,"8000, 8010 and 8020 East Drive North Bay Villa...","(25.8545286, -80.1575787)",25.854529,-80.157579,https://therealdeal.com/wp-content/uploads/202...
4,7918 West Drive,Pacific & Orient Properties,"$8,300,000",54,"Pacific & Orient Properties plans a 21-story, ...",https://therealdeal.com/tag/north-bay-village/,"7918 West Drive North Bay Village, FL","(25.850618, -80.160242)",25.850618,-80.160242,https://therealdeal.com/wp-content/uploads/201...


In [14]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2023/01...
1    <a href="https://therealdeal.com/miami/2022/10...
2    <a href="https://therealdeal.com/miami/2022/09...
3    <a href="https://therealdeal.com/miami/2023/05...
4    <a href="https://therealdeal.com/tag/north-bay...
Name: description_link, dtype: object

In [15]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Developer: </strong>{}'''.format(developer) + '''<br>
    {}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [16]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=12)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'North Bay Village Development Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [17]:
m.save('index.html')

## Map URL Snagger

In [18]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NorthBayVillage-development
